# Data analysis script

## Input
a single file received from running `grim dump` on the payload

## Output
- FILL THIS IN

In [1]:
import struct
import base64
from collections import namedtuple
from typing import Tuple

## Data opening and parsing
- open a file to bytes
- bytes to lists of data

In [2]:
SEPARATOR = "********\n"
filename = 'data/all.cap'

In [3]:
def file_to_bytes(filename: str) -> Tuple[bytes, bytes, bytes, bytes, bytes]:
    with open(filename, 'r') as f:
        parts = f.read().split(SEPARATOR)
        if len(parts) != 5:
            print("wrong number of parts. did you remove all extra lines on top and bottom")
        byte_parts = [base64.b64decode(part) for part in parts]
        return tuple(byte_parts)

In [26]:
SlowData = namedtuple('SlowData', ['timestamp', 'humidity', 'temperature', 'grim_voltage', 'grim_current', 'load_cell_voltage', 'load_cell_current', 'bat_voltage', 'bat_current'])
FastData = namedtuple('FastData', ['timestamp', 'accx', 'accy', 'accz', 'gyrox', 'gyroy', 'gyroz', 'pressure'])
ADCData = namedtuple('ADCData', ['timestamp', 'reading'])
# PreIMUData is FastData
PreALTData = namedtuple('PreALTData', ['timestamp', 'pressure', 'temperature'])


In [81]:
slow_fmt='IffHHHHHH' # timestamp, humid, temp, (voltage, current) * 3
fast_fmt = 'Ifffffff' # timestamp accxyz, gyro xyz, press
adc_fmt = 'Iiiiiiiiiii' # timestamp + 10 int32s
imu_boost_detect_fmt = fast_fmt
alt_boost_detect_fmt = 'Iff' # timestamp, press, temp

In [82]:
slow_bs, fast_bs, adc_bs, pre_imu_bs, pre_alt_bs = file_to_bytes(filename)

In [45]:
pre_alt_bs

b'E\x04\x00\x00\xa9\xfd\xc3BH\xe1\xb2AN\x04\x00\x00\xa9\xfd\xc3BH\xe1\xb2AX\x04\x00\x00\xa9\xfd\xc3BH\xe1\xb2Ab\x04\x00\x00\xa9\xfd\xc3BH\xe1\xb2Am\x04\x00\x00o\xfd\xc3B\xc3\xf5\xb2Av\x04\x00\x00o\xfd\xc3B\xc3\xf5\xb2A\x80\x04\x00\x00o\xfd\xc3B\xc3\xf5\xb2A\x8a\x04\x00\x00J\xfd\xc3B\xc3\xf5\xb2A\x94\x04\x00\x00J\xfd\xc3B\xc3\xf5\xb2A\x9f\x04\x00\x00J\xfd\xc3B\xc3\xf5\xb2A\xa8\x04\x00\x00J\xfd\xc3B\xc3\xf5\xb2A\xb2\x04\x00\x00\x10\xfd\xc3B=\n\xb3A\xbc\x04\x00\x00\x10\xfd\xc3B=\n\xb3A\xc6\x04\x00\x00\x10\xfd\xc3B=\n\xb3A\xd0\x04\x00\x00\x10\xfd\xc3B=\n\xb3A\xda\x04\x00\x00\xa3\xfc\xc3B=\n\xb3A\xe4\x04\x00\x00\xa3\xfc\xc3B=\n\xb3A\xee\x04\x00\x00\xa3\xfc\xc3B=\n\xb3A\xf9\x04\x00\x00\xa3\xfc\xc3B=\n\xb3A\x02\x05\x00\x00\xdc\xfb\xc3B=\n\xb3A\x0c\x05\x00\x00\xdc\xfb\xc3B=\n\xb3A\x16\x05\x00\x00\xdc\xfb\xc3B=\n\xb3A \x05\x00\x00\xdc\xfb\xc3B=\n\xb3A*\x05\x00\x00\xcf\xfb\xc3B\xb8\x1e\xb3A4\x05\x00\x00\xcf\xfb\xc3B\xb8\x1e\xb3A>\x05\x00\x00\xcf\xfb\xc3B\xb8\x1e\xb3A\x00\x00\x00\x00\x00\x00\x00\

In [83]:
slow_lists = list(struct.iter_unpack(slow_fmt, slow_bs))
fast_lists = list(struct.iter_unpack(fast_fmt, fast_bs))
adc_lists = list(struct.iter_unpack(adc_fmt, adc_bs))
pre_imu_lists = list(struct.iter_unpack(imu_boost_detect_fmt, pre_imu_bs))
pre_alt_lists = list(struct.iter_unpack(alt_boost_detect_fmt, pre_alt_bs))


In [84]:
slow_data = [SlowData(*l) for l in slow_lists]
fast_data = [FastData(*l) for l in fast_lists]
# adc_data = interpolate_adc(adc_lists)
# filter out timestamp = 0 (unwritten) entries
# order by timestamp bc the circular buffer may not begin with the earliest entry
pre_imu_data = sorted([FastData(*l) for l in pre_imu_lists if l[0] != 0], key = lambda d : d.timestamp)
pre_alt_data = sorted([PreALTData(*l) for l in pre_alt_lists if l[0] != 0], key = lambda d : d.timestamp)

In [85]:
adc_lists

[(0, -852, -3909, -3909, -4126, -3862, -4022, -3828, -3918, -3960, -3997),
 (2044, -3862, -3850, -3907, -3845, -3887, -3974, -4072, -4050, -3949, -3867),
 (2046, -3794, -4022, -3904, -3926, -3645, -3999, -3912, -4061, -4022, -3867),
 (2048, -3991, -3960, -4022, -4126, -4064, -3901, -3786, -3698, -3850, -3797),
 (2050, -4072, -4050, -3994, -3842, -3867, -3974, -3966, -3907, -3895, -3707),
 (2052, -3892, -3912, -3937, -3929, -3887, -3800, -3904, -4050, -4191, -3968),
 (2054, -3892, -3724, -4019, -3786, -3876, -4095, -4148, -4025, -3853, -3999),
 (2056, -4058, -3833, -3923, -3774, -3968, -4008, -4005, -3901, -3929, -3999),
 (2058, -3971, -3954, -4047, -3952, -3918, -3991, -3786, -3935, -3946, -4039),
 (2061, -3954, -3876, -3772, -3991, -3901, -3966, -3839, -3887, -3788, -3952),
 (2063, -3876, -4047, -4117, -3949, -4044, -3867, -3949, -3884, -3943, -3907),
 (2065, -3907, -3980, -3895, -3847, -4193, -4016, -3929, -3814, -3912, -3966),
 (2067, -4134, -3921, -3831, -4070, -4016, -3943, -4106,